In [1]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('../train_new.csv')
train.head()


,image,class
0,Appelvink_Man_0_frame0.jpg,Appelvink_Man
1,Appelvink_Man_0_frame1.jpg,Appelvink_Man
2,Appelvink_Man_0_frame10.jpg,Appelvink_Man
3,Appelvink_Man_0_frame11.jpg,Appelvink_Man
4,Appelvink_Man_0_frame12.jpg,Appelvink_Man


In [3]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('../training/'+train['class'][i]+'/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████| 9914/9914 [01:53<00:00, 87.17it/s] 


(9914, 224, 224, 3)

In [4]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [5]:
# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

In [6]:
X_train = base_model.predict(X_train)
X_train.shape

(7931, 7, 7, 512)

In [7]:
X_test = base_model.predict(X_test)
X_test.shape

(1983, 7, 7, 512)

In [8]:
X_train = X_train.reshape(7931, 7*7*512)
X_test = X_test.reshape(1983, 7*7*512)

In [9]:
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [10]:
X_train.shape

(7931, 25088)

In [12]:
np.savetxt('../train1.csv', X_train)
np.savetxt('../test1.csv', X_test)


In [13]:
np.savetxt('../y_train1.csv', y_train, fmt='%s')
np.savetxt('../y_test1.csv', y_test, fmt='%s')

In [14]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [28]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(31, activation='softmax'))

In [29]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('../models/weight2.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [30]:

# compiling the model
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [31]:
# training the model
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/50
62/62 [==============================] - 7s 103ms/step - loss: 0.2554 - accuracy: 0.0421 - val_loss: 0.1397 - val_accuracy: 0.1155
Epoch 2/50
62/62 [==============================] - 6s 105ms/step - loss: 0.1587 - accuracy: 0.0803 - val_loss: 0.1357 - val_accuracy: 0.1417
Epoch 3/50
62/62 [==============================] - 6s 104ms/step - loss: 0.1371 - accuracy: 0.1914 - val_loss: 0.1038 - val_accuracy: 0.4125
Epoch 4/50
62/62 [==============================] - 7s 109ms/step - loss: 0.1099 - accuracy: 0.3529 - val_loss: 0.0721 - val_accuracy: 0.5961
Epoch 5/50
62/62 [==============================] - 7s 112ms/step - loss: 0.0866 - accuracy: 0.4876 - val_loss: 0.0515 - val_accuracy: 0.7675
Epoch 6/50
62/62 [==============================] - 6s 102ms/step - loss: 0.0683 - accuracy: 0.6120 - val_loss: 0.0353 - val_accuracy: 0.8487
Epoch 7/50
62/62 [==============================] - 6s 103ms/step - loss: 0.0516 - accuracy: 0.7176 - val_loss: 0.0234 - val_accuracy: 0.8971
Epoch 

KeyboardInterrupt: 